In [1]:
import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./info.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_ = input_[input_['Kit_Part_Number'] != ''].drop_duplicates(['Kit_JOIN_TCPK'],
                                                                 keep='first',
                                                                 ignore_index=True)

print('总数量：' + str(len(input_)))
input_['Kit_Info'].tolist()

总数量：926


['https://www.rockauto.com/en/moreinfo.php?pt=5724&pk=10110008',
 'https://www.rockauto.com/en/moreinfo.php?pt=5724&pk=10110012',
 'https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=10110016',
 'https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=10110020',
 'https://www.rockauto.com/en/moreinfo.php?pt=5736&pk=10110028',
 'https://www.rockauto.com/en/moreinfo.php?pt=5736&pk=10110032',
 'https://www.rockauto.com/en/moreinfo.php?pt=5736&pk=10110036',
 'https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=10110040',
 'https://www.rockauto.com/en/moreinfo.php?pt=5724&pk=8306420',
 'https://www.rockauto.com/en/moreinfo.php?pt=5723&pk=8314716',
 'https://www.rockauto.com/en/moreinfo.php?pt=5736&pk=10110024',
 'https://www.rockauto.com/en/moreinfo.php?pt=10465&pk=10195124',
 'https://www.rockauto.com/en/moreinfo.php?pt=5724&pk=7093400',
 'https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=8307288',
 'https://www.rockauto.com/en/moreinfo.php?pt=5736&pk=8307904',
 'https://www.rockauto.com/en

In [3]:
crawler_status = 'error'

crawler_status

'error'

In [4]:
import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

In [5]:
a = 0

resp = requests.get(input_.loc[a, 'Kit_Info'],
                    headers=UA.get_User_Agent_Requests(),
                    proxies=Proxy.get_Proxy_Requests()).text

resp

'<!DOCTYPE HTML>\n<html class="no-js REPLACEHTMLCLASS" onclick="" lang="en">\n<head>\n<meta name="robots" content="all">\n<meta http-equiv="Content-Security-Policy" content="script-src \'self\' \'unsafe-inline\' \'unsafe-eval\' *.rockauto.com; base-uri \'self\' *.rockauto.com; ">\n<link rel="stylesheet" type="text/css" href="/jslib/combined/css/desktopmoreinfo.css?1718331858">\n\n<script>\nif(!window.RABoot){var no_js="no-js",has_js="has-js",c=document.documentElement.className.replace(no_js,has_js);-1==c.indexOf(has_js)&&(c+=" "+has_js);document.documentElement.className=c;RABoot=function(){var f=function(){};f.prototype={};f.WaitFor=function(a,b){if(_isset(b)&&"function"===typeof b){_isarray(a)||(a=[a]);var d={s:a,w:b,a:{}};CheckReady(d)?d.w():Waiting.push(d)}};_isset=function(a){return"undefined"!=typeof a&&null!=a};_isin=function(a,b){return a in b};_isarray=function(a){return Array.isArray(a)};\n_getref=function(a){var b=null;a=a.split(".");if(0<a.length){"window"==a[0]&&a.shift()

In [6]:
from bs4 import BeautifulSoup
from lxml import etree

In [7]:
# soup = BeautifulSoup(resp)
# html = etree.HTML(str(soup))
html = etree.HTML(str(resp))

# with open('./info.html', 'w', encoding='UTF-8') as file:
#     file.write(str(soup))

# print(soup.prettify())

In [8]:
html.xpath('//span[@class="la-btn-v"]/text()')[0].strip() != 'Continue Shopping'

False

In [9]:
dict_info = {}
list_tr = html.xpath('//table[@class="moreinfotable"]/tr')[1:]
for i in range(len(list_tr)):
    dict_info[str(i)] = {}
    list_td = [str(_).strip() for _ in list_tr[i].xpath('./td/text()')]
    dict_info[str(i)][list_td[0]] = list_td[1]
    
dict_info

{'0': {'Chain Length': '32.250"'},
 '1': {'Chain Length': '819.15 mm'},
 '2': {'Chain Material': 'Hardened Steel'},
 '3': {'Chain Pitch': '9.52 mm'},
 '4': {'Chain Pitch': '0.375"'},
 '5': {'Chain Type': 'Roller'},
 '6': {'Chain Width': '0.540"'},
 '7': {'Chain Width': '13.71 mm'},
 '8': {'Link Quantity': '174'},
 '9': {'Roller Quantity': '1'},
 '10': {'Row Quantity': '1'},
 '11': {'Timing Mark Quantity': '3'},
 '12': {'Timing Marks': 'Yes'}}

In [10]:
import json

In [11]:
info_json = json.dumps(dict_info)

info_json

'{"0": {"Chain Length": "32.250\\""}, "1": {"Chain Length": "819.15 mm"}, "2": {"Chain Material": "Hardened Steel"}, "3": {"Chain Pitch": "9.52 mm"}, "4": {"Chain Pitch": "0.375\\""}, "5": {"Chain Type": "Roller"}, "6": {"Chain Width": "0.540\\""}, "7": {"Chain Width": "13.71 mm"}, "8": {"Link Quantity": "174"}, "9": {"Roller Quantity": "1"}, "10": {"Row Quantity": "1"}, "11": {"Timing Mark Quantity": "3"}, "12": {"Timing Marks": "Yes"}}'

In [12]:
list_tbody = html.xpath('//section[contains(@aria-label, "Components") and contains(@aria-label, "kit")]/table/tbody/tr/td/table/tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-b-0") or contains(@class, "altrow-b-1"))]')

list_tbody

[]

In [13]:
list_quantity = [int(tbody.xpath('./tr/td[1]')[0].text.strip()) for tbody in list_tbody]

list_quantity

[]

In [14]:
list_note = [tbody.xpath('./tr/td[2]')[0].text.strip() for tbody in list_tbody]

list_note

[]

In [15]:
list_json = [json.loads(tbody.xpath('./tr/td[contains(@id, "listing_data_essential")]/@value')[0]) for tbody in list_tbody]

list_json

[]

In [16]:
list_type_code = [json_['parttype'].strip() for json_ in list_json]

list_type_code

[]

In [17]:
list_part_key = [json_['partkey'].strip() for json_ in list_json]

list_part_key

[]

In [18]:
list_json_2 = [json.loads(tbody.xpath('./tr/input[contains(@id, "listing_data_supplemental")]/@value')[0]) for tbody in list_tbody]

list_json_2

[]

In [19]:
list_manufacturer = [json_['catalogname'].strip() for json_ in list_json_2]

list_manufacturer

[]

In [20]:
list_part_number = [json_['partnumber'].strip() for json_ in list_json_2]

list_part_number

[]

In [21]:
dict_kit = {}
for i in range(len(list_tbody)):
    dict_kit[str(i)] = {}
    dict_kit[str(i)]['0'] = list_quantity[i]
    dict_kit[str(i)]['1'] = list_note[i]
    dict_kit[str(i)]['2'] = list_manufacturer[i]
    dict_kit[str(i)]['3'] = list_part_number[i]
    dict_kit[str(i)]['4'] = list_type_code[i]
    dict_kit[str(i)]['5'] = list_part_key[i]
        
dict_kit

{}

In [22]:
kit_json = json.dumps(dict_kit)

kit_json

'{}'

In [23]:
df_temp = pd.DataFrame([{'Kit_Info': input_.loc[a, 'Kit_Info'],
                         'Kit_Specifications_Json': info_json,
                         'Kit_Kit_Json': kit_json}])

for tr in list_tr:
    df_temp.loc[0, tr.xpath('./td[1]')[0].text.lower().strip()] = tr.xpath('./td[2]')[0].text.strip()

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

len(output_correct)

1

In [24]:
crawler_status = 'ok'

crawler_status

'ok'

In [25]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'Kit_Info': input_.loc[a, 'Kit_Info']}])
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

len(output_error)

0

In [26]:
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Kit_Info'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./test_kit_info.xlsx', index=False)

output_correct

,Kit_Info,Kit_Specifications_Json,Kit_Kit_Json,chain length,chain material,chain pitch,chain type,chain width,link quantity,roller quantity,row quantity,timing mark quantity,timing marks
0,https://www.rockauto.com/en/moreinfo.php?pt=57...,"{""0"": {""Chain Length"": ""32.250\""""}, ""1"": {""Cha...",{},819.15 mm,Hardened Steel,"0.375""",Roller,13.71 mm,174,1,1,3,Yes


In [27]:
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Kit_Info'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./test_kit_info_error.xlsx', index=False)

output_error

""
